In [ ]:
# instalar as dependências
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,228 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,561 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,646 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

In [ ]:
import os
os.environ["JAVA"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK"] = "/content/spark-3.5.0-bin-hadoop3"

In [ ]:
# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.5.0-bin-hadoop3')

In [5]:
# Etapa 1
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from google.colab import files
from google.colab import drive

drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab_arquivos/"
file_path = '/content/drive/My Drive/Colab_arquivos/nomes_aleatorios.txt'

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Exercicio Intro") \
    .getOrCreate()


Mounted at /content/drive
nomes_aleatorios.txt


In [7]:
# Etapa 1
df_nomes=spark.read.csv(path=file_path,header=False,inferSchema=True)
df_nomes.show(10)

+-----------------+
|              _c0|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [8]:
df_nomes.printSchema()

root
 |-- _c0: string (nullable = true)



In [9]:
#Etapa 2
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [10]:
#Etapa 3
from pyspark.sql.functions import when, rand

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when((rand() < 0.33), "Fundamental")
    .when((rand() >= 0.33) & (rand() < 0.66), "Medio")
    .otherwise("Superior")
)

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet| Fundamental|
|    Jamie Russell|       Medio|
|   Edward Kistler| Fundamental|
|    Sheila Maurer| Fundamental|
| Donald Golightly|    Superior|
|       David Gray|       Medio|
|      Joy Bennett|    Superior|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|    Superior|
|    Brian Farrell|    Superior|
+-----------------+------------+
only showing top 10 rows



In [11]:
# Etapa 4
paises = ["Argentina", "Bolívia", "Brasil", "Chile", "Colômbia", "Equador", "Guiana",
          "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela", "Guiana Francesa"]
df_nomes = df_nomes.withColumn(
    "Pais",
    when(rand() < 1/13, paises[0])
    .when((rand() >= 1/13) & (rand() < 2/13), paises[1])
    .when((rand() >= 2/13) & (rand() < 3/13), paises[2])
    .when((rand() >= 3/13) & (rand() < 4/13), paises[3])
    .when((rand() >= 4/13) & (rand() < 5/13), paises[4])
    .when((rand() >= 5/13) & (rand() < 6/13), paises[5])
    .when((rand() >= 6/13) & (rand() < 7/13), paises[6])
    .when((rand() >= 7/13) & (rand() < 8/13), paises[7])
    .when((rand() >= 8/13) & (rand() < 9/13), paises[8])
    .when((rand() >= 9/13) & (rand() < 10/13), paises[9])
    .when((rand() >= 10/13) & (rand() < 11/13), paises[10])
    .when((rand() >= 11/13) & (rand() < 12/13), paises[11])
    .otherwise(paises[12])
)

df_nomes.show(10)

+-----------------+------------+---------------+
|            Nomes|Escolaridade|           Pais|
+-----------------+------------+---------------+
|   Frances Bennet| Fundamental|Guiana Francesa|
|    Jamie Russell|       Medio|        Bolívia|
|   Edward Kistler| Fundamental|        Bolívia|
|    Sheila Maurer| Fundamental|      Argentina|
| Donald Golightly|    Superior|        Equador|
|       David Gray|       Medio|      Argentina|
|      Joy Bennett|    Superior|        Bolívia|
|      Paul Kriese| Fundamental|        Bolívia|
|Berniece Ornellas|    Superior|       Colômbia|
|    Brian Farrell|    Superior|          Chile|
+-----------------+------------+---------------+
only showing top 10 rows



In [12]:
# Etapa 5
from pyspark.sql.functions import col, floor

df_nomes = df_nomes.withColumn(
    "AnoNascimento",
    (floor(rand() * (2010 - 1945 + 1)) + 1945).cast("int")
)

df_nomes.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet| Fundamental|Guiana Francesa|         1954|
|    Jamie Russell|       Medio|        Bolívia|         1971|
|   Edward Kistler| Fundamental|        Bolívia|         1991|
|    Sheila Maurer| Fundamental|      Argentina|         1974|
| Donald Golightly|    Superior|        Equador|         1968|
|       David Gray|       Medio|      Argentina|         1975|
|      Joy Bennett|    Superior|        Bolívia|         1998|
|      Paul Kriese| Fundamental|        Bolívia|         1947|
|Berniece Ornellas|    Superior|       Colômbia|         2006|
|    Brian Farrell|    Superior|          Chile|         1997|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



In [13]:
# Etapa 6
df_select=df_nomes.filter((col("AnoNascimento")>=2001))
df_select.show(10)

+--------------------+------------+---------+-------------+
|               Nomes|Escolaridade|     Pais|AnoNascimento|
+--------------------+------------+---------+-------------+
|   Berniece Ornellas|    Superior| Colômbia|         2006|
|      Kara Mcelwaine|       Medio|  Bolívia|         2009|
|         Frank Wiley|       Medio|   Guiana|         2006|
|     Kenneth Rayburn| Fundamental|   Guiana|         2006|
|          Anita Ross|       Medio|   Brasil|         2002|
|      Mary Dillahunt|       Medio|    Chile|         2003|
|         Sandra Todd|    Superior|    Chile|         2007|
|       Ricky Gilbert|       Medio|  Bolívia|         2007|
|Christopher Williams|    Superior|    Chile|         2010|
|     Charles Randall|    Superior|Argentina|         2001|
+--------------------+------------+---------+-------------+
only showing top 10 rows



In [14]:
# Etapa 7
df_nomes.createOrReplaceTempView("tabela_nomes")
df_seculo_atual_sql = spark.sql("""
    SELECT *
    FROM tabela_nomes
    WHERE AnoNascimento >= 2001
""")
df_seculo_atual_sql.show(10)

+--------------------+------------+---------+-------------+
|               Nomes|Escolaridade|     Pais|AnoNascimento|
+--------------------+------------+---------+-------------+
|   Berniece Ornellas|    Superior| Colômbia|         2006|
|      Kara Mcelwaine|       Medio|  Bolívia|         2009|
|         Frank Wiley|       Medio|   Guiana|         2006|
|     Kenneth Rayburn| Fundamental|   Guiana|         2006|
|          Anita Ross|       Medio|   Brasil|         2002|
|      Mary Dillahunt|       Medio|    Chile|         2003|
|         Sandra Todd|    Superior|    Chile|         2007|
|       Ricky Gilbert|       Medio|  Bolívia|         2007|
|Christopher Williams|    Superior|    Chile|         2010|
|     Charles Randall|    Superior|Argentina|         2001|
+--------------------+------------+---------+-------------+
only showing top 10 rows



In [15]:
# Etapa 8
df_millennials=df_nomes.filter((col("AnoNascimento")>=1980) & (col("AnoNascimento")<=1994))
total=df_millennials.count()
print(f"Pessoas que são da geração millennials: {total}")

Pessoas que são da geração millennials: 2275499


In [16]:
# Etapa 9
n_pessoas = spark.sql("""
    SELECT COUNT(*) AS total_pessoas
    FROM tabela_nomes
    WHERE AnoNascimento >= 1980 AND AnoNascimento <= 1994
""")
n_pessoas.show()

+-------------+
|total_pessoas|
+-------------+
|      2275499|
+-------------+



In [17]:
# Etapa 10
resultado = spark.sql("""
    SELECT
        Pais,
        CASE
            WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
            WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
            WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
            WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
            ELSE 'Outros'
        END AS Geracao,
        COUNT(*) AS Quantidade
    FROM tabela_nomes
    WHERE AnoNascimento BETWEEN 1944 AND 2015
    GROUP BY Pais, Geracao
    ORDER BY Pais ASC, Geracao ASC, Quantidade ASC
""")

resultado.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    232803|
|Argentina|   Geração X|    174083|
|Argentina|   Geração Z|    185853|
|Argentina| Millennials|    175101|
|  Bolívia|Baby Boomers|    397076|
|  Bolívia|   Geração X|    297252|
|  Bolívia|   Geração Z|    317154|
|  Bolívia| Millennials|    298633|
|   Brasil|Baby Boomers|    468059|
|   Brasil|   Geração X|    350948|
|   Brasil|   Geração Z|    374620|
|   Brasil| Millennials|    351516|
|    Chile|Baby Boomers|    456528|
|    Chile|   Geração X|    343018|
|    Chile|   Geração Z|    365795|
|    Chile| Millennials|    343547|
| Colômbia|Baby Boomers|    393457|
| Colômbia|   Geração X|    294159|
| Colômbia|   Geração Z|    314784|
| Colômbia| Millennials|    295226|
+---------+------------+----------+
only showing top 20 rows

